# Spark SQL - house stats

in folder `data/` there is a file `rollingsales_bronx.json`.

Let's analyze statistical data for houses: counts, avarege areas, years, prices grouping by hood and type.

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as func


spark = SparkSession \
    .builder \
    .appName("FlightData") \
    .config("master", "local[*]") \
    .getOrCreate()

In [ ]:
flight_data = spark.read.parquet('../resources/data/flight-data/parquet/2010-summary.parquet/part-r-00000-1a9822ba-b8fb-4d8e-844a-ea30d0801b9e.gz.parquet')
# Register table
flight_data.registerTempTable('flight_data')
flight_data.printSchema()
flight_data.show()

In [ ]:
flightData2015_csv = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv("../resources/data/flight-data/csv/2015-summary.csv")

In [ ]:
flightData2015_csv.take(5)

In [ ]:
flightData2015_csv.sort("count").explain()

In [ ]:
flightData2015_csv.sort("count").take(2)

In [ ]:
flightData2015_csv.createOrReplaceTempView("flight_data_2015")

In [ ]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
maxSql.show()

In [ ]:
from pyspark.sql.functions import desc

flightData2015_csv\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.show()